Import data.

In [ ]:
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, classification_report, coverage_error, label_ranking_loss, \
    label_ranking_average_precision_score, zero_one_loss
from sklearn.multioutput import ClassifierChain
from sklearn.datasets import make_multilabel_classification
from sklearn.svm import LinearSVC

#load data
from google.colab import files
uploaded = files.upload()
import io
warnings.filterwarnings("ignore")
X_test = pd.read_csv(io.BytesIO(uploaded['test-data.dat']))
X_train =pd.read_csv(io.BytesIO(uploaded['train-data.dat']))
Y_test = pd.read_csv(io.BytesIO(uploaded['test-label.dat']))
Y_train = pd.read_csv(io.BytesIO(uploaded['train-label.dat']))

Y_test_new = Y_test.values
Y_train_new= Y_train.values


Saving test-data.dat to test-data (2).dat
Saving test-label.dat to test-label (2).dat
Saving train-data.dat to train-data (2).dat
Saving train-label.dat to train-label (2).dat


Preprocess the label data (Y_test, Y_train) to properly print the classification_report later.

In [ ]:
array_list_test = []
for index, row in Y_test.iterrows():
    split_values = (row[0])  # Split the numbers into a list of strings
    filtered_list = [item for item in split_values if item.strip() in ('0', '1')]
    int_array = np.array([int(num) for num in filtered_list])
    array_list_test.append(int_array)
merged_array_test = np.array(array_list_test)

array_list_train = []
for index, row in Y_train.iterrows():
    split_values = (row[0])  # Split the numbers into a list of strings
    filtered_list = [item for item in split_values if item.strip() in ('0', '1')]
    int_array = np.array([int(num) for num in filtered_list])
    array_list_train.append(int_array)
merged_array_train = np.array(array_list_train)

Y_test_new = merged_array_test
Y_train_new= merged_array_train

Remove the <> tags and the chars enclosed in them - we are only concerned with words here.

In [ ]:
# remove the <> tags, they are not needed
X_train = X_train.replace(r'<.*?>', '', regex=True)
X_test = X_test.replace(r'<.*?>', '', regex=True)

vectorizer = TfidfVectorizer()
X_train_array = X_train.values
X_test_array = X_test.values

#vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train_array.ravel())
X_test = vectorizer.transform(X_test_array.ravel())

Run the multi-label classification models.

In [ ]:
names = ['LR', 'SVM']
classifiers = [ClassifierChain(LogisticRegression(random_state=123)),
               ClassifierChain(LinearSVC(max_iter=300))
               ]

for name, clf in zip(names, classifiers):
    print(name)
    clf.fit(X_train, Y_train_new)
    Y_pred = clf.predict(X_test)

    print(classification_report(Y_test_new, Y_pred, zero_division='warn'))
    print('subset accuracy   : %.2f' % (1 - zero_one_loss(Y_test_new, Y_pred)))

    ranking_loss = 1 - label_ranking_average_precision_score(Y_test_new, Y_pred)
    avg_precision = label_ranking_average_precision_score(Y_test_new, Y_pred)

    if name == 'LR':
        Y_proba = clf.predict_proba(X_test)
        print('coverage error    : %.2f' % (coverage_error(Y_test_new, Y_proba)))
        print('ranking loss      : %.2f' % (label_ranking_loss(Y_test_new, Y_proba)))
        print('average precision : %.2f' % (label_ranking_average_precision_score(Y_test_new, Y_proba)))

LR
              precision    recall  f1-score   support

           0       0.84      0.58      0.69       976
           1       0.94      0.14      0.24       227
           2       0.67      0.39      0.49      1557
           3       0.81      0.47      0.60       372
           4       0.70      0.36      0.48      1050
           5       0.48      0.15      0.22       537
           6       0.61      0.12      0.20       702
           7       0.76      0.26      0.39      1079
           8       0.72      0.30      0.43       803
           9       0.79      0.27      0.40       483
          10       0.73      0.21      0.33       507
          11       0.86      0.13      0.23       478
          12       0.60      0.09      0.15       509
          13       0.70      0.17      0.27       355
          14       0.76      0.24      0.36       392
          15       0.72      0.06      0.12       441
          16       0.66      0.09      0.16       269
          17       0.87 

In summary, the SVM model generally outperforms the LR model in terms of precision, recall, F1-score, and subset accuracy. However, the LR model has considerably higher average precision and fares well in terms of ranking loss.